In [1]:
import sys
import uuid
from pyspark.sql.functions import struct, col, to_json
from lib import ConfigLoader, Utils, DataLoader, Transformations
from lib.logger import Log4j

In [2]:
# Definindo o ambiente de execução e a data de carregamento
# Se não for LOCAL, outras configurações são aplicadas ao cluster Spark
job_run_env = "LOCAL"  
load_date = "2024-10-05"  

# Gerando um identificador único para esta execução do job
job_run_id = "SBDL-" + str(uuid.uuid4())  

# Imprime no console o início da execução do job
print(f"Initializing SBDL Job in {job_run_env} Job ID: {job_run_id}")

# Carrega as configurações do arquivo correspondente ao ambiente (ex.: conf/local.conf)
conf = ConfigLoader.get_config(job_run_env)

# Verifica se o Hive está habilitado nas configurações, para decidir se será utilizado
enable_hive = True if conf["enable.hive"] == "true" else False
hive_db = conf["hive.database"]  # Obtém o nome do banco de dados Hive das configurações

# Inicializando a sessão Spark, essencial para processamento distribuído de dados
print("Creating Spark Session")
spark = Utils.get_spark_session(job_run_env)  # Cria a sessão Spark apropriada ao ambiente

# Inicializa o logger para registrar informações no log
logger = Log4j(spark)

Initializing SBDL Job in LOCAL Job ID: SBDL-a71e1c8e-6296-4185-bed8-932f2914f813
Creating Spark Session
:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9d8194d1-8504-45f1-8cbc-ede56137d9ce;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report 

24/10/05 22:44:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Carrega os dados das contas do sistema (DataFrame de contas)
logger.info("Reading SBDL Account DF")
accounts_df = DataLoader.read_accounts(spark, job_run_env, enable_hive, hive_db)
accounts_df.show(5)  
accounts_df.printSchema()  

24/10/05 22:44:16 INFO sbdl: Reading SBDL Account DF


+----------+----------+----------+----------+-------------------+---------------+--------------+-----------+------------------+-----------+-------------+
| load_date|active_ind|account_id|source_sys| account_start_date|  legal_title_1| legal_title_2|tax_id_type|            tax_id|branch_code|      country|
+----------+----------+----------+----------+-------------------+---------------+--------------+-----------+------------------+-----------+-------------+
|2022-08-02|         1|6982391060|       COH|2018-03-24 08:26:45|  Tiffany Riley|Matthew Davies|        EIN|ZLCK91795330413525|   ACXMGBA5|       Mexico|
|2022-08-02|         1|6982391061|       ADS|2018-07-19 05:54:49|Garcia and Sons| Taylor Guzman|        SSP|CADU39916151090321|   SHJFGBML|United States|
|2022-08-02|         1|6982391067|       BDL|2018-08-29 11:48:59|     Acosta Inc|  David Walker|        SSP|UJLN20870916345792|   WZTEGBTG|       Canada|
|2022-08-02|         1|6982391064|       ADS|2018-03-28 10:17:43|    Allen G

In [4]:
# Aplica transformações para criar um DataFrame de contratos a partir das contas
contract_df = Transformations.get_contract(accounts_df)
contract_df.show(5)
contract_df.printSchema()

+----------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|account_id|  contractIdentifier|sourceSystemIdentifier|contactStartDateTime|       contractTitle|       taxIdentifier|  contractBranchCode|     contractCountry|
+----------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|6982391060|{INSERT, 69823910...|   {INSERT, COH, null}|{INSERT, 2018-03-...|{INSERT, [{lgl_tt...|{INSERT, {EIN, ZL...|{INSERT, ACXMGBA5...|{INSERT, Mexico, ...|
|6982391061|{INSERT, 69823910...|   {INSERT, ADS, null}|{INSERT, 2018-07-...|{INSERT, [{lgl_tt...|{INSERT, {SSP, CA...|{INSERT, SHJFGBML...|{INSERT, United S...|
|6982391067|{INSERT, 69823910...|   {INSERT, BDL, null}|{INSERT, 2018-08-...|{INSERT, [{lgl_tt...|{INSERT, {SSP, UJ...|{INSERT, WZTEGBTG...|{INSERT, Canada, ...|
|6982391064|{INSERT, 6982391

In [5]:
# Carrega os dados das partes envolvidas (DataFrame de clientes ou entidades relacionadas)
logger.info("Reading SBDL Party DF")
parties_df = DataLoader.read_parties(spark, job_run_env, enable_hive, hive_db)
parties_df.show(5)
parties_df.printSchema()

24/10/05 22:44:28 INFO sbdl: Reading SBDL Party DF
+----------+----------+----------+-------------+-------------------+
| load_date|account_id|  party_id|relation_type|relation_start_date|
+----------+----------+----------+-------------+-------------------+
|2022-08-02|6982391060|9823462810|          F-N|2019-07-29 00:51:32|
|2022-08-02|6982391061|9823462811|          F-N|2018-08-30 23:57:22|
|2022-08-02|6982391062|9823462812|          F-N|2018-08-25 10:20:29|
|2022-08-02|6982391063|9823462813|          F-N|2018-05-11 01:53:28|
|2022-08-02|6982391064|9823462814|          F-N|2019-06-06 08:48:12|
+----------+----------+----------+-------------+-------------------+
only showing top 5 rows

root
 |-- load_date: date (nullable = true)
 |-- account_id: string (nullable = true)
 |-- party_id: string (nullable = true)
 |-- relation_type: string (nullable = true)
 |-- relation_start_date: timestamp (nullable = true)



In [6]:
# Aplica transformações para obter as relações entre essas partes
relations_df = Transformations.get_relations(parties_df)
relations_df.show(5)
relations_df.printSchema()

+----------+----------+--------------------+---------------------+--------------------------+
|account_id|  party_id|     partyIdentifier|partyRelationshipType|partyRelationStartDateTime|
+----------+----------+--------------------+---------------------+--------------------------+
|6982391060|9823462810|{INSERT, 98234628...|  {INSERT, F-N, null}|      {INSERT, 2019-07-...|
|6982391061|9823462811|{INSERT, 98234628...|  {INSERT, F-N, null}|      {INSERT, 2018-08-...|
|6982391062|9823462812|{INSERT, 98234628...|  {INSERT, F-N, null}|      {INSERT, 2018-08-...|
|6982391063|9823462813|{INSERT, 98234628...|  {INSERT, F-N, null}|      {INSERT, 2018-05-...|
|6982391064|9823462814|{INSERT, 98234628...|  {INSERT, F-N, null}|      {INSERT, 2019-06-...|
+----------+----------+--------------------+---------------------+--------------------------+
only showing top 5 rows

root
 |-- account_id: string (nullable = true)
 |-- party_id: string (nullable = true)
 |-- partyIdentifier: struct (nullable = f

In [7]:
# Carrega os dados de endereços (DataFrame de endereços)
logger.info("Reading SBDL Address DF")
address_df = DataLoader.read_address(spark, job_run_env, enable_hive, hive_db)
address_df.show(5)
address_df.printSchema()

24/10/05 22:44:29 INFO sbdl: Reading SBDL Address DF
+----------+----------+------------------+--------------------+--------------+-----------+------------------+------------------+
| load_date|  party_id|    address_line_1|      address_line_2|          city|postal_code|country_of_address|address_start_date|
+----------+----------+------------------+--------------------+--------------+-----------+------------------+------------------+
|2022-08-02|9823462810| 45229 Drake Route|   13306 Corey Point|     Shanefort|      77163|            Canada|        2019-02-26|
|2022-08-02|9823462811|361 Robinson Green|3511 Rebecca Mission|   North Tyler|      34118|            Canada|        2018-01-28|
|2022-08-02|9823462812|  039 Daniel Mount|8219 Hernandez Lo...| Boltonborough|      71648|            Mexico|        2018-12-07|
|2022-08-02|9823462813|05550 Nancy Rapids| 9471 Zachary Canyon|East Davidport|      02504|     United States|        2019-04-02|
|2022-08-02|9823462814| 5227 Wagner Pines|18

In [8]:
# Aplica transformações para obter o DataFrame de endereços relacionados
relation_address_df = Transformations.get_address(address_df)
relation_address_df.show(5)
relation_address_df.printSchema()

+----------+--------------------+
|  party_id|        partyAddress|
+----------+--------------------+
|9823462810|{INSERT, {45229 D...|
|9823462811|{INSERT, {361 Rob...|
|9823462812|{INSERT, {039 Dan...|
|9823462813|{INSERT, {05550 N...|
|9823462814|{INSERT, {5227 Wa...|
+----------+--------------------+
only showing top 5 rows

root
 |-- party_id: string (nullable = true)
 |-- partyAddress: struct (nullable = false)
 |    |-- operation: string (nullable = false)
 |    |-- newValue: struct (nullable = false)
 |    |    |-- addressLine1: string (nullable = true)
 |    |    |-- addressLine2: string (nullable = true)
 |    |    |-- addressCity: string (nullable = true)
 |    |    |-- addressPostalCode: string (nullable = true)
 |    |    |-- addressCountry: string (nullable = true)
 |    |    |-- addressStartDate: date (nullable = true)
 |    |-- oldValue: void (nullable = true)



In [9]:
# Junta os dados de relações de partes com os endereços para criar um DataFrame único
logger.info("Join Party Relations and Address")
party_address_df = Transformations.join_party_address(relations_df, relation_address_df)
party_address_df.show(5)
party_address_df.printSchema()

24/10/05 22:44:31 INFO sbdl: Join Party Relations and Address


+----------+--------------------+
|account_id|      partyRelations|
+----------+--------------------+
|6982391067|[{{INSERT, 982346...|
|6982391064|[{{INSERT, 982346...|
|6982391066|[{{INSERT, 982346...|
|6982391061|[{{INSERT, 982346...|
|6982391060|[{{INSERT, 982346...|
+----------+--------------------+
only showing top 5 rows

root
 |-- account_id: string (nullable = true)
 |-- partyRelations: array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- partyIdentifier: struct (nullable = false)
 |    |    |    |-- operation: string (nullable = false)
 |    |    |    |-- newValue: string (nullable = true)
 |    |    |    |-- oldValue: void (nullable = true)
 |    |    |-- partyRelationshipType: struct (nullable = false)
 |    |    |    |-- operation: string (nullable = false)
 |    |    |    |-- newValue: string (nullable = true)
 |    |    |    |-- oldValue: void (nullable = true)
 |    |    |-- partyRelationStartDateTime: struct (nullable = false)
 |    

In [10]:
# Junta os dados de contratos com os dados de partes e endereços
logger.info("Join Account and Parties")
data_df = Transformations.join_contract_party(contract_df, party_address_df)
data_df.show(5)
data_df.printSchema()

24/10/05 22:44:39 INFO sbdl: Join Account and Parties


+----------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|account_id|  contractIdentifier|sourceSystemIdentifier|contactStartDateTime|       contractTitle|       taxIdentifier|  contractBranchCode|     contractCountry|      partyRelations|
+----------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|6982391067|{INSERT, 69823910...|   {INSERT, BDL, null}|{INSERT, 2018-08-...|{INSERT, [{lgl_tt...|{INSERT, {SSP, UJ...|{INSERT, WZTEGBTG...|{INSERT, Canada, ...|[{{INSERT, 982346...|
|6982391064|{INSERT, 69823910...|   {INSERT, ADS, null}|{INSERT, 2018-03-...|{INSERT, [{lgl_tt...|{INSERT, {CPR, WJ...|{INSERT, OCCKGB65...|{INSERT, Canada, ...|[{{INSERT, 982346...|
|6982391066|{INSERT, 69823910...|   {INSERT, BDL, null}|{INSERT, 2017-08-...|{INSERT,

In [11]:
# Aplica um cabeçalho e cria um evento (provavelmente adiciona metadados ou estrutura padrão)
logger.info("Apply Header and create Event")
final_df = Transformations.apply_header(spark, data_df)
final_df.show(5)
final_df.printSchema()

24/10/05 22:44:48 INFO sbdl: Apply Header and create Event


+--------------------+--------------------+--------------------+
|         eventHeader|                keys|             payload|
+--------------------+--------------------+--------------------+
|{962276d7-2609-47...|[{contractIdentif...|{{INSERT, 6982391...|
|{b4c4af05-23a9-45...|[{contractIdentif...|{{INSERT, 6982391...|
|{c6a493b7-db6e-46...|[{contractIdentif...|{{INSERT, 6982391...|
|{d9d9a906-22e3-43...|[{contractIdentif...|{{INSERT, 6982391...|
|{4f11e2d7-10c3-4f...|[{contractIdentif...|{{INSERT, 6982391...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

root
 |-- eventHeader: struct (nullable = false)
 |    |-- eventIdentifier: string (nullable = false)
 |    |-- eventType: string (nullable = true)
 |    |-- majorSchemaVersion: long (nullable = true)
 |    |-- minorSchemaVersion: long (nullable = true)
 |    |-- eventDateTime: string (nullable = false)
 |-- keys: array (nullable = false)
 |    |-- element: struct (containsNull = false)
